In [1]:
# IMPORT PACKAGES
import os
import numpy as np
from skimage import filters, measure, morphology, io
from skimage.segmentation import clear_border
from aicsimageio import AICSImage
import cv2
import pandas as pd
import time
from datetime import datetime, timedelta

# TO SUPPRESS WARNINGS BEING PRINTED
import warnings
warnings.filterwarnings("ignore")

/camp/home/davise/home/users/davise/conda/envs/lazemap_env/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
# SET PARAMETERS FOR ANALYSIS
c1gaussRad = 2.0
c2gaussRad = 2.0
c1thresholdMethod = "isodata"
c2thresholdMethod = "isodata"

In [3]:
# DIRECTORY SET UP
main_dir = "/camp/home/davise/home/shared/thackrm/Batch_processing_images/"
image_dir = os.path.join(main_dir, "Images")
results_dir = os.path.join(main_dir, "results")
sense_check_dir = os.path.join(main_dir, "sense_check")

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
if not os.path.exists(sense_check_dir):
    os.makedirs(sense_check_dir)

fileList = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))
            and f.endswith('.czi')]
print(f"{len(fileList)} files found")

# LIST FOR ALL RESULTS TABLE
all_results = []


3 files found


In [4]:
def process_image(filePath):
    start_time = time.time()
    
    extractedName = extract_file_name(filePath)
    print(f"Processing: {extractedName}")

    # CREATE SENSE CHECK DIRECTORY FOR CURRENT IMG
    image_sense_check_dir = os.path.join(sense_check_dir, extractedName)
    if not os.path.exists(image_sense_check_dir):
        os.makedirs(image_sense_check_dir)
    
    img = AICSImage(filePath)
    data = img.get_image_data("CZYX", S=0, T=0)
    
    # GET INFO TO CONVERT FROM PIXELS TO VOXELS
    voxel_size = img.physical_pixel_sizes  # Get voxel dimensions (z, y, x) in microns
    voxel_volume = voxel_size.Z * voxel_size.Y * voxel_size.X  # Calculate voxel volume in cubic microns
    
    # SPLIT CHANNELS
    c1 = data[0, :, :, :]
    c2 = data[1, :, :, :]

    # SAVE ORIG IMAGES AS SENSE CHECK
    for z in range(0, c1.shape[0], 50):  # Save every 50th slice
        io.imsave(os.path.join(image_sense_check_dir, f"c1_orig_z{z}.png"), c1[z, :, :])
        io.imsave(os.path.join(image_sense_check_dir, f"c2_orig_z{z}.png"), c2[z, :, :])
    
    # GAUSSIAN BLUR USING OPENCV
    c1_blurred = np.array([cv2.GaussianBlur(c1[z, :, :], (0, 0), c1gaussRad) for z in range(c1.shape[0])])
    c2_blurred = np.array([cv2.GaussianBlur(c2[z, :, :], (0, 0), c2gaussRad) for z in range(c2.shape[0])])
    
    # SAVE BLUR IMAGES AS SENSE CHECK
    for z in range(0, c1_blurred.shape[0], 50):
        io.imsave(os.path.join(image_sense_check_dir, f"c1_blurred_z{z}.png"), c1_blurred[z, :, :])
        io.imsave(os.path.join(image_sense_check_dir, f"c2_blurred_z{z}.png"), c2_blurred[z, :, :])
    
    # THRESHOLD
    c1_binary = np.array([c1_blurred[z, :, :] > filters.threshold_isodata(c1_blurred[z, :, :]) for z in range(c1_blurred.shape[0])])
    c2_binary = np.array([c2_blurred[z, :, :] > filters.threshold_isodata(c2_blurred[z, :, :]) for z in range(c2_blurred.shape[0])])
        
    # SAVE BINARY IMAGES AS SENSE CHECK
    for z in range(0, c1_binary.shape[0], 50):
        io.imsave(os.path.join(image_sense_check_dir, f"c1_binary_z{z}.png"), c1_binary[z, :, :].astype(np.uint8) * 255)
        io.imsave(os.path.join(image_sense_check_dir, f"c2_binary_z{z}.png"), c2_binary[z, :, :].astype(np.uint8) * 255)
    
    # CALC. OVERLAP
    overlap = np.logical_and(c1_binary, c2_binary)
    
    # SAVE OVERLAP AS SENSE CHECK
    for z in range(0, overlap.shape[0], 50):
        io.imsave(os.path.join(image_sense_check_dir, f"overlap_z{z}.png"), overlap[z, :, :].astype(np.uint8) * 255)
    
    # LABEL OBJECTS AND CALC. VOLUME
    c1_labels = measure.label(c1_binary, connectivity=1)
    c2_labels = measure.label(c2_binary, connectivity=1)
    overlap_labels = measure.label(overlap, connectivity=1)
    
    # SAVE LABEL IMAGES AS SENSE CHECK
    for z in range(0, c1_labels.shape[0], 50):
        io.imsave(os.path.join(image_sense_check_dir, f"c1_labels_z{z}.png"), c1_labels[z, :, :].astype(np.uint16))
        io.imsave(os.path.join(image_sense_check_dir, f"c2_labels_z{z}.png"), c2_labels[z, :, :].astype(np.uint16))
        io.imsave(os.path.join(image_sense_check_dir, f"overlap_labels_z{z}.png"), overlap_labels[z, :, :].astype(np.uint16))
    
    c1_props = measure.regionprops(c1_labels)
    c2_props = measure.regionprops(c2_labels)
    overlap_props = measure.regionprops(overlap_labels)
    
    c1_volume = sum([prop.area for prop in c1_props]) * voxel_volume
    c2_volume = sum([prop.area for prop in c2_props]) * voxel_volume
    overlap_volume = sum([prop.area for prop in overlap_props]) * voxel_volume
    
    print(f"Channel 1 volume: {c1_volume}")
    print(f"Channel 2 volume: {c2_volume}")
    print(f"Overlap volume: {overlap_volume}")
    print("\n")
    
    # Compile results into a list
    all_results.append({
        "Image": extractedName,
        "Channel": "Channel 1",
        "Volume (microns^3)": c1_volume,
        "% Overlap": 100.0 * (overlap_volume / c1_volume) if c1_volume != 0 else np.nan
    })
    all_results.append({
        "Image": extractedName,
        "Channel": "Channel 2",
        "Volume (microns^3)": c2_volume,
        "% Overlap": 100.0 * (overlap_volume / c2_volume) if c2_volume != 0 else np.nan
    })
    all_results.append({
        "Image": extractedName,
        "Channel": "Overlap",
        "Volume (microns^3)": overlap_volume,
        "% Overlap": np.nan
    })
    
    elapsed_time = time.time() - start_time
    return elapsed_time

def extract_file_name(filePath):
    fileName = os.path.basename(filePath)
    fileNameWithoutExt = os.path.splitext(fileName)[0]
    return fileNameWithoutExt

In [ ]:
total_images = len(fileList)
total_time = 0

# APPLY TO EACH IMAGE IN LIST
for idx, file in enumerate(fileList):
    elapsed_time = process_image(os.path.join(image_dir, file))
    total_time += elapsed_time
    
    # CALC. AND PRINT ETA
    remaining_images = total_images - (idx + 1)
    avg_time_per_image = total_time / (idx + 1)
    eta_seconds = remaining_images * avg_time_per_image
    eta = datetime.now() + timedelta(seconds=eta_seconds)
    print(f"Processed {idx + 1}/{total_images} images. ETA all images finished: {eta.strftime('%H:%M')}")

# STORE ALL RESULTS IN ONE CSV
all_results_df = pd.DataFrame(all_results)
all_results_df.to_csv(os.path.join(results_dir, "compiled_results.csv"), index=False)


Processing: BDAU7_2c-04-Airyscan_Processing-02
Channel 1 volume: 37672.607163102155
Channel 2 volume: 4452.128245535758
Overlap volume: 1028.035091008305


Processed 1/3 images. ETA all images finished: 16:00
Processing: BDAU7_2c-08-Airyscan_Processing-03
Channel 1 volume: 30772.44441698522
Channel 2 volume: 4682.689762959804
Overlap volume: 621.4379247180667


Processed 2/3 images. ETA all images finished: 16:00
Processing: BDAU7_2c-01-Airyscan_Processing-01
